In [1]:
import numpy as np
import pywt

import seaborn as sns #绘制confusion matrix heatmap

import sklearn
import os

import warnings

warnings.simplefilter('ignore') #忽略警告

In [2]:
import scipy
import scipy.io as sio

import scipy.signal

from scipy import linalg

import pandas as pd
from sklearn.decomposition import PCA
#分类器
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split 

import xgboost
import lightgbm

#模型集成
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from mlxtend.classifier import StackingClassifier

#模型调节
from sklearn.model_selection import GridSearchCV #参数搜索
from mlxtend.feature_selection import SequentialFeatureSelector #特征选择函数 选择合适的feature

#结果可视化
from sklearn.metrics import classification_report , confusion_matrix #混淆矩阵

#相关指标
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.metrics import cohen_kappa_score

from sklearn.metrics import roc_auc_score

#二分类其多分类化
#from sklearn.multiclass import OneVsOneClassifier
#from sklearn.multiclass import OneVsRestClassifier

#from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import KMeans

#距离函数 度量向量距离
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity #余弦相似度

#one-hot使用
#from keras.utils import to_categorical

from sklearn.preprocessing import label_binarize

#绘图
import matplotlib.pyplot as plt

import scipy.linalg as la

import gc

%matplotlib inline

In [3]:
sample_rate = 256 #hz
origin_channel = 16 #5 channel eeg

#采集的通道
#共16 channel
#未使用的channel使用none代替
#reference:a study on performance increasing in ssvep based bci application
SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']


# 减去前多少秒数据 second
# 减去后多少秒数据 second
CLIP_FORWARD = 2
CLIP_BACKWARD = 1

# 单个小段的实验时长
trial_time = 3 #second

trial_offset = 0 #second
start_trial_time = 0 #真正的实验开始时刻
end_trial_time = 2 #真正的实验结束时刻(<trial_time)

#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频



In [4]:
def butter_worth(data , lowcut , highcut , order=6):
    nyq = 0.5 * sample_rate
    
    lo = lowcut / nyq
    hi = highcut / nyq
    
    b,a = scipy.signal.butter(order , [lo , hi] , btype='bandpass')

    return np.array([scipy.signal.filtfilt(b , a , data[: , i]) for i in range(data.shape[1])]).reshape((-1 , origin_channel))

def load_data(filename):    
    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix
    
    #截掉前后数据
    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate]
    
    #滤波
    #return butter_worth(data , 0.5 , 30)

    return data

In [5]:
def separate(data , label , overlap_length):
    '''
    最长重叠长度为size长 256*3 个数据点
    '''
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]

    idx = 0

    while idx<data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)

    return np.array(train_data) , np.array(train_labels)

In [6]:
def shuffle_t_v(filenames):
    np.random.shuffle(filenames)
    
    return filenames

def combine(freq):
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('data/0/%s/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('data/1/%s/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('data/2/%s/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('data/3/%s/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('data/4/%s/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('data/5/%s/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('data/6/%s/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('data/7/%s/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('data/8/%s/' % freq) )

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('data/0/%s/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('data/1/%s/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('data/2/%s/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('data/3/%s/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('data/4/%s/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('data/5/%s/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('data/6/%s/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('data/7/%s/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('data/8/%s/' % freq + filename) for filename in person_8_filenames] , axis = 0)
        
    #============
    #训练数据分段
    person_data_0 , person_labels_0 = separate(person_0 , label = 0 , overlap_length=overlap_length)
    person_data_1 , person_labels_1 = separate(person_1 , label = 1 , overlap_length=overlap_length)
    person_data_2 , person_labels_2 = separate(person_2 , label = 2 , overlap_length=overlap_length)
    person_data_3 , person_labels_3 = separate(person_3 , label = 3 , overlap_length=overlap_length)
    person_data_4 , person_labels_4 = separate(person_4 , label = 4 , overlap_length=overlap_length)
    person_data_5 , person_labels_5 = separate(person_5 , label = 5 , overlap_length=overlap_length)
    person_data_6 , person_labels_6 = separate(person_6 , label = 6 , overlap_length=overlap_length)
    person_data_7 , person_labels_7 = separate(person_7 , label = 7 , overlap_length=overlap_length)
    person_data_8 , person_labels_8 = separate(person_8 , label = 8 , overlap_length=overlap_length)

    #合并数据
    data = np.concatenate((person_data_0 , person_data_1 , person_data_2 ,
                           person_data_3 , person_data_4 , person_data_5 ,
                           person_data_6 , person_data_7 , person_data_8 
                        ))
    
    labels = np.concatenate((person_labels_0 , person_labels_1 , person_labels_2 ,
                             person_labels_3 , person_labels_4 , person_labels_5 ,
                             person_labels_6 , person_labels_7 , person_labels_8 
                        ))
    
    return data , labels

In [7]:
def feature_extraction_dwt_meta(data , n):
    n_features = 48
    
    X = np.zeros((data.shape[0] , n_features))
    
    level = 5
    wavelet = 'db4'
    
    #n=3 or 4
    
    for i , datum in enumerate(data):
        coeffs_Pz  = pywt.wavedec(data = datum[:,0], wavelet=wavelet, level=level)
        coeffs_PO3 = pywt.wavedec(data = datum[:,1], wavelet=wavelet, level=level)
        coeffs_PO4 = pywt.wavedec(data = datum[:,2], wavelet=wavelet, level=level)
        coeffs_O1  = pywt.wavedec(data = datum[:,3], wavelet=wavelet, level=level)
        coeffs_O2  = pywt.wavedec(data = datum[:,4], wavelet=wavelet, level=level)
        coeffs_Oz  = pywt.wavedec(data = datum[:,5], wavelet=wavelet, level=level)
        coeffs_O9  = pywt.wavedec(data = datum[:,6], wavelet=wavelet, level=level)
        coeffs_FP2 = pywt.wavedec(data = datum[:,7], wavelet=wavelet, level=level)
        coeffs_C4  = pywt.wavedec(data = datum[:,8], wavelet=wavelet, level=level)
        coeffs_C6  = pywt.wavedec(data = datum[:,9], wavelet=wavelet, level=level)
        coeffs_CP3 = pywt.wavedec(data = datum[:,10], wavelet=wavelet, level=level)
        coeffs_CP1 = pywt.wavedec(data = datum[:,11], wavelet=wavelet, level=level)
        coeffs_CPZ = pywt.wavedec(data = datum[:,12], wavelet=wavelet, level=level)
        coeffs_CP2 = pywt.wavedec(data = datum[:,13], wavelet=wavelet, level=level)
        coeffs_CP4 = pywt.wavedec(data = datum[:,14], wavelet=wavelet, level=level)
        coeffs_PO8 = pywt.wavedec(data = datum[:,15], wavelet=wavelet, level=level)

        X[i , :] = np.array([
            np.std(coeffs_Pz [n]),   
            np.std(coeffs_PO3[n]),  
            np.std(coeffs_PO4[n]),   
            np.std(coeffs_O1 [n]),  
            np.std(coeffs_O2 [n]),  
            np.std(coeffs_Oz [n]),   
            np.std(coeffs_O9 [n]),  
            np.std(coeffs_FP2[n]),  
            np.std(coeffs_C4 [n]),
            np.std(coeffs_C6 [n]),
            np.std(coeffs_CP3[n]),
            np.std(coeffs_CP1[n]),
            np.std(coeffs_CPZ[n]),
            np.std(coeffs_CP2[n]),
            np.std(coeffs_CP4[n]),
            np.std(coeffs_PO8[n]),
            
            np.mean(coeffs_Pz [n]**2),
            np.mean(coeffs_PO3[n]**2),
            np.mean(coeffs_PO4[n]**2),
            np.mean(coeffs_O1 [n]**2),
            np.mean(coeffs_O2 [n]**2),
            np.mean(coeffs_Oz [n]**2),
            np.mean(coeffs_O9 [n]**2),
            np.mean(coeffs_FP2[n]**2),            
            np.mean(coeffs_C4 [n]**2),
            np.mean(coeffs_C6 [n]**2),
            np.mean(coeffs_CP3[n]**2),
            np.mean(coeffs_CP1[n]**2),
            np.mean(coeffs_CPZ[n]**2),
            np.mean(coeffs_CP2[n]**2),
            np.mean(coeffs_CP4[n]**2),
            np.mean(coeffs_PO8[n]**2),
            
            np.mean(coeffs_Pz [n]),
            np.mean(coeffs_PO3[n]), 
            np.mean(coeffs_PO4[n]),
            np.mean(coeffs_O1 [n]),
            np.mean(coeffs_O2 [n]),
            np.mean(coeffs_Oz [n]),
            np.mean(coeffs_O9 [n]),
            np.mean(coeffs_FP2[n]),        
            np.mean(coeffs_C4 [n]),
            np.mean(coeffs_C6 [n]),
            np.mean(coeffs_CP3[n]),
            np.mean(coeffs_CP1[n]),
            np.mean(coeffs_CPZ[n]),
            np.mean(coeffs_CP2[n]),
            np.mean(coeffs_CP4[n]),
            np.mean(coeffs_PO8[n])]).flatten()
        
    return X

def normalize(data , normalization_type = 'mean_std'):
    
    def _norm_mean_std(data):
        _mean = np.mean(data , axis=0)
        _std = np.std(data , axis=0)
        
        return (data - _mean) / _std
    
    def _norm_min_max(data):
        return (data - np.min(data)) / (np.max(data) - np.min(data))
    
    if normalization_type == 'mean_std':
        return _norm_mean_std(data)
    elif normalization_type == 'min_max':
        return _norm_min_max(data)
    else:
        raise Exception('wrong normalization type')
    
def feature_extraction_dwt(data , is_normalize = True):
    data_3 = feature_extraction_dwt_meta(data , 3) #4
    data_4 = feature_extraction_dwt_meta(data , 4) #5
    
    data_concat = np.concatenate((data_3 , data_4) , axis = -1)
    
    if is_normalize:
        return normalize(data_concat)
    else:
        return data_concat

# decision tree boosting

In [8]:
acc_s = []

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data_feature = feature_extraction_dwt(data) #特征提取
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(DecisionTreeClassifier() , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.9814323607427056
0.9854111405835544
0.9787798408488063
0.9880636604774535
0.9787798408488063
0.9748010610079576
0.9734748010610079
0.9721485411140584
0.9721485411140584
0.9787798408488063
0.9787798408488063
0.9694960212201591
0.980106100795756
0.9854111405835544
0.9748010610079576
0.9854111405835544
0.9681697612732095
0.980106100795756
0.9721485411140584
0.9814323607427056
freq =  6 average_acc =  0.9779840848806366 var_acc =  3.067635739363539e-05
freq =  7.5
0.9787798408488063
0.986737400530504
0.9880636604774535
0.9840848806366048
0.9854111405835544
0.9748010610079576
0.9814323607427056
0.9814323607427056
0.980106100795756
0.9774535809018567
0.9827586206896551
0.96684350132626
0.9854111405835544
0.9787798408488063
0.9748010610079576
0.9774535809018567
0.9774535809018567
0.980106100795756
0.9761273209549072
0.980106100795756
freq =  7.5 average_acc =  0.9799071618037136 var_acc =  2.3090818903953387e-05
freq =  8.5
0.9748010610079576
0.9761273209549072
0.9708222811671088


# svm boosting

In [8]:
acc_s = []

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data_feature = feature_extraction_dwt(data) #特征提取
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=SVC(probability=True) , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.23076923076923078
0.23076923076923078
0.2891246684350133
0.22281167108753316
0.2625994694960212
0.27984084880636606
0.2639257294429708
0.2427055702917772
0.25596816976127323
0.24005305039787797
0.26657824933687
0.26657824933687
0.28249336870026526
0.2493368700265252
0.2639257294429708
0.2984084880636605
0.27718832891246686
0.2917771883289125
0.28779840848806365
0.27984084880636606
freq =  6 average_acc =  0.26412466843501325 var_acc =  0.0004698680424121749
freq =  7.5
0.27718832891246686
0.2427055702917772
0.27851458885941643
0.28249336870026526
0.20689655172413793
0.2413793103448276
0.2440318302387268
0.23342175066312998
0.2917771883289125
0.28514588859416445
0.22413793103448276
0.2612732095490716
0.29045092838196285
0.24005305039787797
0.259946949602122
0.2493368700265252
0.20954907161803712
0.2639257294429708
0.2546419098143236
0.2493368700265252
freq =  7.5 average_acc =  0.2543103448275862 var_acc =  0.0005997852303189356
freq =  8.5
0.2427055702917772
0.3116710875331

# naive bayes boosting

In [8]:
acc_s = []

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data_feature = feature_extraction_dwt(data) #特征提取
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = AdaBoostClassifier(base_estimator=GaussianNB() , n_estimators=20)

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.5994694960212201
0.6419098143236074
0.6379310344827587
0.5742705570291777
0.5676392572944297
0.5954907161803713
0.6896551724137931
0.4350132625994695
0.47480106100795755
0.5026525198938993
0.5875331564986738
0.6458885941644562
0.5437665782493368
0.53315649867374
0.623342175066313
0.6047745358090185
0.6074270557029178
0.6671087533156499
0.5716180371352785
0.5159151193633952
freq =  6 average_acc =  0.5809681697612732 var_acc =  0.004049222009582845
freq =  7.5
0.47214854111405835
0.6525198938992043
0.610079575596817
0.6803713527851459
0.46419098143236076
0.596816976127321
0.629973474801061
0.4854111405835544
0.5543766578249337
0.5424403183023873
0.5782493368700266
0.6259946949602122
0.5477453580901857
0.669761273209549
0.5954907161803713
0.6989389920424404
0.6007957559681698
0.6472148541114059
0.596816976127321
0.5848806366047745
freq =  7.5 average_acc =  0.591710875331565 var_acc =  0.004159649860338144
freq =  8.5
0.6671087533156499
0.7188328912466844
0.6127320954907162
0